In [8]:
from typing import List

import sqlite3 as sqlite
import pandas as pd
import json

#importando todos os arquivos csv
country : pd.DataFrame = pd.read_csv('../csv_files/Country.csv')
league : pd.DataFrame = pd.read_csv('../csv_files/League.csv') 
match : pd.DataFrame = pd.read_csv('../csv_files/Match.csv') 
player_attributes : pd.DataFrame = pd.read_csv('../csv_files/Player_Attributes.csv',index_col = 0) 
player : pd.DataFrame = pd.read_csv('../csv_files/Player.csv') 
team_attributes : pd.DataFrame = pd.read_csv('../csv_files/Team_Attributes.csv',index_col = 0) 
team : pd.DataFrame = pd.read_csv('../csv_files/Team.csv') 
    
#criando um banco de dados
#como não há no teste uma definição para o nome do database eu utilizei clicksign_test.db
conn = sqlite.connect('clicksign_test.db')
cursor = conn.cursor()

#Criando uma função que registra as tabelas no schema
#O padrão registrado está em conformidade com a documentação sqlite : https://www.sqlite.org/fileformat2.html#ffschema
def schema_update(schema_type : str, name : str, tbl_name : str, rootpage : int, sql : str):
        cursor.execute("INSERT INTO test_analytics_engineer VALUES (?,?,?,?,?);",
                       [schema_type, name, tbl_name, rootpage, sql])

#Criando o schema test_analytics_engineer de acordo com a documentação do sqlite
cursor.executescript("""
    DROP TABLE IF EXISTS test_analytics_engineer;
    CREATE TABLE test_analytics_engineer (type, name, tbl_name, rootpage, sql);
""")

#Registrando as tabelas no schema, materializando no db e fazendo carga dos dados
#tabela country
#Configurando o schema
schema_type : str = 'table'
name : str = 'table'
tbl_name : str = 'country'
rootpage : int = 1
sql : str = 'CREATE TABLE'
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela e fazendo a carga
cursor.executescript("""
    DROP TABLE IF EXISTS country;
    CREATE TABLE country (id, name);
""")
insert_query = 'INSERT INTO country VALUES (?,?)'
cursor.executemany(insert_query, country.values.tolist()) 

#Tabela league
#Configurando o schema
tbl_name : str = 'league'
rootpage : int =+ 1
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela e fazendo a carga
cursor.executescript("""
    DROP TABLE IF EXISTS league;
    CREATE TABLE league (id, country_id, name);
""")
insert_query = 'INSERT INTO league VALUES (?,?,?)'
cursor.executemany(insert_query, league.values.tolist())

#Tabela match
#Configurando o schema
tbl_name : str = 'match'
rootpage : int =+ 1
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela e fazendo a carga
cursor.executescript("""
    DROP TABLE IF EXISTS match;

    CREATE TABLE match (
        id, country_id, league_id, season, 
        stage, date, match_api_id, home_team_api_id, 
        away_team_api_id, home_team_goal, away_team_goal, 
        home_player_X1, home_player_X2, home_player_X3,
        home_player_X4, home_player_X5, home_player_X6, 
        home_player_X7, home_player_X8, home_player_X9, 
        home_player_X10, home_player_X11, 
        away_player_X1, away_player_X2, away_player_X3, 
        away_player_X4, away_player_X5, away_player_X6, 
        away_player_X7, away_player_X8, away_player_X9, 
        away_player_X10, away_player_X11,
        home_player_Y1, home_player_Y2, home_player_Y3, 
        home_player_Y4, home_player_Y5, home_player_Y6, 
        home_player_Y7, home_player_Y8, home_player_Y9, 
        home_player_Y10, home_player_Y11, 
        away_player_Y1, away_player_Y2, away_player_Y3, 
        away_player_Y4, away_player_Y5, away_player_Y6,
        away_player_Y7, away_player_Y8, away_player_Y9, 
        away_player_Y10, away_player_Y11, 
        home_player_1, home_player_2, home_player_3, 
        home_player_4, home_player_5, home_player_6, 
        home_player_7, home_player_8, home_player_9, 
        home_player_10, home_player_11, 
        away_player_1, away_player_2, away_player_3, 
        away_player_4, away_player_5, away_player_6, 
        away_player_7, away_player_8, away_player_9, 
        away_player_10, away_player_11,
        goal, shoton, shotoff, foulcommit, card, cross, corner, possession, 
        B365H, B365D, B365A, BWH, BWD, BWA,
        IWH, IWD, IWA, 
        LBH, LBD, LBA, 
        PSH, PSD, PSA, 
        WHH, WHD, WHA, 
        SJH, SJD, SJA, 
        VCH, VCD, VCA, 
        GBH, GBD, GBA, 
        BSH, BSD, BSA 
    );
""")

insert_query = '''INSERT INTO match VALUES (
                     ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                     ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                     ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                     ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                     ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                     ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''

cursor.executemany(insert_query, match.values.tolist())

#tabela player_attributes
#Configurando o schema
tbl_name : str = 'player_attributes'
rootpage : int =+ 1
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela e fazendo a carga
cursor.executescript("""
    DROP TABLE IF EXISTS player_attributes;
    CREATE TABLE player_attributes(player_attributes_json json);
""")
for i in  player_attributes.values.tolist():
    i = json.loads(i[0])
    cursor.execute('INSERT INTO player_attributes VALUES (?);' , [json.dumps(i)])

#tabela player
#Configurando o schema
tbl_name : str = 'player'
rootpage : int =+ 1
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela e fazendo a carga
cursor.executescript("""
    DROP TABLE IF EXISTS player;
    CREATE TABLE player(id, player_api_id, player_name, player_fifa_api_id, birthday, height, weight);
""")
insert_query = 'INSERT INTO player VALUES (?,?,?,?,?,?,?);'
cursor.executemany(insert_query, player.values.tolist())

#tabela team_attributes
#Configurando o schema
tbl_name : str = 'team_attributes'
rootpage : int =+ 1
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela
cursor.executescript("""
    DROP TABLE IF EXISTS team_attributes;
    CREATE TABLE team_attributes(team_attributes json);
""")
#Fazendo o upload
for i in  team_attributes.values.tolist():
    i = json.loads(i[0])
    cursor.execute('INSERT INTO team_attributes VALUES (?);' , [json.dumps(i)])

#tabela team
#Configurando o schema
tbl_name : str = 'team'
rootpage : int =+ 1
schema_update(schema_type, name, tbl_name, rootpage, sql)

#Criando a tabela e fazendo a carga
cursor.executescript("""
    DROP TABLE IF EXISTS team;
    CREATE TABLE team(id, team_api_id, team_fifa_api_id, team_long_name, team_short_name);
""")
insert_query = 'INSERT INTO team VALUES (?,?,?,?,?);'
cursor.executemany(insert_query, team.values.tolist())

#Finalizando a conexão
conn.commit()
conn.close()